# Tabu Search



Approach
    - The aim of this script is to calculate VIF and determine which features to consider for model training
    - No extensive cleaning, outlier, or trend outlier consideration will be done
        - The focus is VIF, not neccesarily model performance

In [40]:
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import ipywidgets as widgets
import random as rand

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# Greedy Algorithm

In [41]:
# Take original feature list, and replace one feature with a random other number not in that list
def get_neighbor_random_insert_greedy(initial_features, potential_features):
    
    # Generate a random feature from the potential features list not in the initial features list
    rand_feature = potential_features[rand.randint(0, len(potential_features)-1)]
    while True:
        if rand_feature in initial_features:
            rand_feature = potential_features[rand.randint(0, len(potential_features)-1)]
        else:
            # If not in initial_features, select random position to insert feature
            rand_pos = rand.randint(0, len(initial_features)-1)
            initial_features[rand_pos] = rand_feature
            break
    return initial_features


Ingestion

In [42]:
df_optimization = pd.read_csv("df_pre_transform.csv")
df_optimization = df_optimization.drop("Unnamed: 0", axis=1)

# preparing response variable data
target = df_optimization.loc[:, 'REVENUE']
all_features = df_optimization.loc[:, df_optimization.columns.isin(['REVENUE']) == False]

# preparing training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
            all_features,
            target,
            test_size = 0.25,
            random_state = 222)

In [43]:
# Initialization
train = X_train
train_score = []
test_score = []
sig_features = []
model_type = []

# p-value interations
for i in range(0, 5, 1):
    
    # Find good p-values from OLS
    model = sm.OLS(y_train, train).fit()
    significant_features = list(model.pvalues[model.pvalues < 0.05].index.values)
    
    # Test and train with scikit-learn
    lr = LinearRegression().fit(X_train.loc[:, significant_features], y_train)

    lr_train_score = lr.score(X_train.loc[:, significant_features], y_train).round(4)
    lr_test_score = lr.score(X_test.loc[:, significant_features], y_test).round(4)

    # storing data
    train_score.append(lr_train_score)
    test_score.append(lr_test_score)
    sig_features.append(significant_features)
    model_type.append("LR")
    # train = X_train.loc[:, model.pvalues[model.pvalues < 0.05].index.values]

model_performance = pd.DataFrame(
    {
        "model_type": model_type, 
        "train_score": train_score, 
        "test_score": test_score, 
        "features": sig_features
    })

In [44]:
p_val_features = model_performance.iloc[4, :].features
model_performance

,model_type,train_score,test_score,features
0,LR,0.6798,0.6098,"[TOTAL_MEALS_ORDERED, UNIQUE_MEALS_PURCH, CUST..."
1,LR,0.6798,0.6098,"[TOTAL_MEALS_ORDERED, UNIQUE_MEALS_PURCH, CUST..."
2,LR,0.6798,0.6098,"[TOTAL_MEALS_ORDERED, UNIQUE_MEALS_PURCH, CUST..."
3,LR,0.6798,0.6098,"[TOTAL_MEALS_ORDERED, UNIQUE_MEALS_PURCH, CUST..."
4,LR,0.6798,0.6098,"[TOTAL_MEALS_ORDERED, UNIQUE_MEALS_PURCH, CUST..."


Greedy search with p_value_ optimized columns

In [45]:
# Setup greedy algorithm
iterations = 1000
max_score = 0
best_train = []
best_test = []
best_features = []

# first iteration of p-value features
features_considered = p_val_features

# regress on features specific to p-values
X_tr = X_train.loc[:, features_considered]
X_te = X_test.loc[:, features_considered]

# train test score
lr = LinearRegression().fit(X_tr, y_train)
train_score = lr.score(X_tr, y_train)
test_score = lr.score(X_te, y_test)

# max criteria
if test_score > max_score:
    max_score = test_score
    best_train.append(train_score)
    best_test.append(max_score)
#     print(round(max_score, 4))
    best_features.append(features_considered)
        
for i in range(0, iterations, 1):
    
    # Generate a random greedy neighbor
    initial_greedy_random_neighbor_index = get_feature_index_locations(all_features, p_val_features)
    potential_other_greedy_features = get_potential_features(all_features, initial_greedy_random_neighbor_index)
    features_considered = get_neighbor_random_insert_greedy(initial_greedy_random_neighbor_index, potential_other_greedy_features)

    # regress on features specific to p-values
    X_tr = X_train.iloc[:, features_considered]
    X_te = X_test.iloc[:, features_considered]
    
    # train test score
    lr = LinearRegression().fit(X_tr, y_train)
    train_score = lr.score(X_tr, y_train)
    test_score = lr.score(X_te, y_test)
    
    # max criteria
    if test_score > max_score:
        max_score = test_score
        best_train.append(train_score)
        best_test.append(max_score)
#         print(round(max_score, 4))
        best_features.append(features_considered)

In [46]:
print("p-value features - Test: {}".format(
    LinearRegression()\
    .fit(X_train.loc[:, p_val_features], y_train)\
    .score(X_test.loc[:, p_val_features], y_test)))

print("Greedy search with p-value features: {}".format(
    LinearRegression()\
    .fit(X_train.iloc[:, best_features[-1]], y_train)\
    .score(X_test.iloc[:, best_features[-1]], y_test)))

p-value features - Test: 0.6097774098551809
Greedy search with p-value features: 0.6154690025645886


# Tabu Search Algorithm

## Tabu Search implementation

In [47]:
# [1,3] in [[1,2], [4, 5]]

# get list of neighbors that are not considered for optimization
def get_neighbors(initial_features, potential_features):
    # initialize data structures
    current_neighbor = []
    neighborhood_list = []
    swap_list = [] # []
    
    # decide a random potential feature
    feature_index_swap = rand.randint(0, len(potential_features)-1)
    
    # generate neighborhood swaps
    for i in range(0, len(initial_features), 1):
        
        # Shadow copy of initial features
        current_neighbor = initial_features[:]
        current_neighbor[i] = potential_features[feature_index_swap]
        neighborhood_list.append(current_neighbor)
        swap_list.append([i, feature_index_swap])
        
    return neighborhood_list, swap_list
        

def get_potential_features(features_all, initial_best_neighbor_index):
    potential_index = []
    for i in features_all:
        if features_all.columns.get_loc(i) not in initial_best_neighbor_index:
            potential_index.append(features_all.columns.get_loc(i))
    return potential_index

def get_feature_index_locations(df, column_features):
    feature_indexes = []
    for i in column_features:
        if i in df.columns:
            feature_indexes.append(df.columns.get_loc(i))
    return feature_indexes

In [50]:
algorithm_comp_count = 0

# algorithm setup
initial_best_neighbor_index = get_feature_index_locations(all_features, p_val_features)
potential_other_features_index = get_potential_features(all_features, initial_best_neighbor_index)
best_solution_index = initial_best_neighbor_index[:]
lr = LinearRegression()
tabu_list_index = []
best_neighbor_score = 0 
best_neighbor_features = []
best_neighbor_score_l = []
best_neighbor_features_l = []

best_global_score = 0
best_global_score_l = []
best_global_features_l = []

# algorithm parameter setup
tabu_tenure = 5
aspire_iteration = 1000

for i in range(0, aspire_iteration, 1):
    algorithm_comp_count += 1
    
    # generate neighborhood space
    neighbors, neighbor_swaps = get_neighbors(initial_best_neighbor_index, potential_other_features_index)
    
    for i in range(0, len(neighbors), 1):

        # In this case, a feature swap of 1 and 5 is the same as 5 and 1
        neighbor_to_swap = neighbor_swaps[i]
        alt_neighbor_to_swap = neighbor_to_swap.reverse()

        # check tabu list
        if (neighbor_to_swap) not in tabu_list_index:
            if (alt_neighbor_to_swap) not in tabu_list_index:
                # Calculate algorithm score
                lr_fit = lr.fit(X_train.iloc[:, neighbors[i]], y_train)
                lr_train_score = lr.score(X_train.iloc[:, neighbors[i]], y_train).round(4)
                lr_test_score = lr.score(X_test.iloc[:, neighbors[i]], y_test).round(4)

                # if neighbor score is greater then best score, update initial_best_neighbor_index
                if lr_test_score > best_neighbor_score:
                    best_neighbor_score = lr_test_score
                    best_neighbor_features = neighbors[i]
                    best_swap = neighbor_to_swap
                    initial_best_neighbor_index = neighbors[i]

    # Update Tabu List, best neighbor and corresponding features
    if len(tabu_list_index) < 10:
        tabu_list_index.append(best_swap)
    else:
        # if tabu list is greater then tabu_tenure, then push all values up list by 1 and rid that last entry
        tabu_list_index[1:] = tabu_list_index[0:-1]
        tabu_list_index[0] = best_swap
        
#     print(tabu_list_index)
    
    best_neighbor_score_l.append(best_neighbor_score)
    best_neighbor_features_l.append(best_neighbor_features)
    
    # Update global maximum scores
    if best_neighbor_score > best_global_score:
        best_global_score = best_neighbor_score
        best_global_score_l.append(best_neighbor_score)
        best_global_features_l.append(best_neighbor_features)
        aspire_iteration = 1
    else:
        aspire_iteration += 1
        # print("TS aspired..")
        
    # Reset neighborhood scores
    best_neighbor_score = 0
    lr_train_score = 0
    lr_test_score = 0
    
print("Total iterations: {}".format(algorithm_comp_count))
# print("")
# print(best_global_score_l[-1])
# print(best_global_features_l[-1])

Total iterations: 1000


In [51]:
print("p-value features - Test: {}".format(
    LinearRegression()\
    .fit(X_train.loc[:, p_val_features], y_train)\
    .score(X_test.loc[:, p_val_features], y_test)))

print("Tabu search with p-value features: {}".format(
    LinearRegression()\
    .fit(X_train.iloc[:, best_global_features_l[-1]], y_train)\
    .score(X_test.iloc[:, best_global_features_l[-1]], y_test)))

p-value features - Test: 0.6097774098551809
Tabu search with p-value features: 0.6215748156564418


In [29]:
best_global_score

0.6216

In [56]:
len(best_global_features_l[-1])

17

In [55]:
len(p_val_features)

17